In [8]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from pylab import rcParams
from scipy.ndimage.interpolation import rotate, zoom
import time

In [28]:
imgPaths = glob.glob("../imgs/*/w_*")

In [ ]:
pathAug = "augmented"
if not os.path.exists(pathAug):
    os.makedirs(pathAug)
labels = glob.glob("../imgs/whale_*/lS_*.jpg")
nObs = len(labels)
print("Have %d labels"%nObs)

egImg = Image.open(labels[0])
ar = (egImg.size[0]/float(egImg.size[1]))
newWidth = 300
newHeight = int(300/ar)
print("reshaping to %d by %d" %(newWidth,newHeight))


rcParams['figure.figsize'] = 9, ar*9

In [34]:
def resizeImages(labeledOrOriginal):

    s = str(labeledOrOriginal)
    imgPaths = glob.glob("../imgs/*/"+s+"_*")
    for imgPath in imgPaths:
        
        idx = imgPaths.index(imgPath)
      
        img = Image.open(imgPath)
        img = img.resize((newWidth,newHeight),Image.ANTIALIAS)
        dst = imgPath.replace(s+"_",s+"S_")
        if idx % 100 == 0:
            print(idx,len(imgPaths))
            #print(dst)
        img.save(dst)
#resizeImages("l") # downscale labels
resizeImages("w") # downscale originals

0 11468
../imgs/test/wS_0.jpg
100 11468
../imgs/test/wS_10094.jpg
200 11468
../imgs/test/wS_10189.jpg
300 11468
../imgs/test/wS_10280.jpg
400 11468
../imgs/test/wS_10375.jpg
500 11468
../imgs/test/wS_10473.jpg
600 11468
../imgs/test/wS_10569.jpg
700 11468
../imgs/test/wS_10662.jpg
800 11468
../imgs/test/wS_10759.jpg
900 11468
../imgs/test/wS_10854.jpg
1000 11468
../imgs/test/wS_10948.jpg
1100 11468
../imgs/test/wS_11039.jpg
1200 11468
../imgs/test/wS_11133.jpg
1300 11468
../imgs/test/wS_1123.jpg
1400 11468
../imgs/test/wS_11323.jpg
1500 11468
../imgs/test/wS_11417.jpg
1600 11468
../imgs/test/wS_1223.jpg
1700 11468
../imgs/test/wS_1407.jpg
1800 11468
../imgs/test/wS_1574.jpg
1900 11468
../imgs/test/wS_1729.jpg
2000 11468
../imgs/test/wS_1898.jpg
2100 11468
../imgs/test/wS_2064.jpg
2200 11468
../imgs/test/wS_224.jpg
2300 11468
../imgs/test/wS_2386.jpg
2400 11468
../imgs/test/wS_255.jpg
2500 11468
../imgs/test/wS_2712.jpg
2600 11468
../imgs/test/wS_2889.jpg
2700 11468
../imgs/test/wS_3058

In [25]:
averageTimes = []
imgNo = 0
nImgs = len(labels)
nAug = 20
for label in labels[:]:
    y = label
    x = y.replace("lS_","wS_")
    imgX, imgY = Image.open(x), Image.open(y)
    
    r = np.array(imgY)
    notRed = (r[:,:,0] < 200) | (r[:,:,1] > 100) | (r[:,:,2] > 100)
    r[notRed] = 0
    red = np.invert(notRed)
    r[red] = 255
    r = Image.fromarray(r)
    
    for i in range(nAug):
        start = time.time()
        angle = np.random.uniform(-30,30)
        zoomFactor = np.random.uniform(0.9,1.1)
        imgX_, r_ = imgX.copy(), r.copy()
        if np.random.uniform() < 0.5:
            imgX_, r_ = [img.transpose(Image.FLIP_LEFT_RIGHT) for img in [imgX_,r_]]
        imgX_, r_ = [rotate(img,angle,mode="nearest") for img in [imgX_,r_]]
        imgX_, r_ = [Image.fromarray(zoom(img,zoomFactor)) for img in [imgX_,r_]]
        imgX_, r_ = [img.resize((newWidth,newHeight)) for img in [imgX_,r_]]
        
        imgX_.save("augmented/x_"+str(imgNo)+".png")
        r_.save("augmented/y_"+str(imgNo)+".png")
        
        elapsed = time.time() - start
        averageTimes.append(elapsed)
        imgNo += 1
        if imgNo % 20 == 0:
            print("%d of %d" %(imgNo,nAug*nImgs))
        def show():
            if np.random.uniform()< 1:
                plt.imshow(np.concatenate((imgX_,r_),axis=1))
                plt.show()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:568: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


20 of 3480
40 of 3480
60 of 3480
80 of 3480
100 of 3480
120 of 3480
140 of 3480
160 of 3480
180 of 3480
200 of 3480
220 of 3480
240 of 3480
260 of 3480
280 of 3480
300 of 3480
320 of 3480
340 of 3480
360 of 3480
380 of 3480
400 of 3480
420 of 3480
440 of 3480
460 of 3480
480 of 3480
500 of 3480
520 of 3480
540 of 3480
560 of 3480
580 of 3480
600 of 3480
620 of 3480
640 of 3480
660 of 3480
680 of 3480
700 of 3480
720 of 3480
740 of 3480
760 of 3480
780 of 3480
800 of 3480
820 of 3480
840 of 3480
860 of 3480
880 of 3480
900 of 3480
920 of 3480
940 of 3480
960 of 3480
980 of 3480
1000 of 3480
1020 of 3480
1040 of 3480
1060 of 3480
1080 of 3480
1100 of 3480
1120 of 3480
1140 of 3480
1160 of 3480
1180 of 3480
1200 of 3480
1220 of 3480
1240 of 3480
1260 of 3480
1280 of 3480
1300 of 3480
1320 of 3480
1340 of 3480
1360 of 3480
1380 of 3480
1400 of 3480
1420 of 3480
1440 of 3480
1460 of 3480
1480 of 3480
1500 of 3480
1520 of 3480
1540 of 3480
1560 of 3480
1580 of 3480
1600 of 3480
1620 of 3480
